In [1]:
import numpy as np

In [72]:
size = 9
states = np.array([[1, 3, 0], [4, size - 2, 1], [size - 2, 3, 2], [2, 1, 3], [3, 3, 2], [4, 4, 0], [1, 1, 3], [2, 4, 0]])
actions = np.array([0, 1, 2, 0, 1, 2, 1, 0])
direction_mapping = np.array([[-1, 0], [0, 1], [1, 0], [0, -1]], dtype=np.int16)
state_max = [size - 2, size - 2, 3]
state_min = [1, 1, 0]

In [73]:
states.shape[0]

8

In [74]:

north_facing = states[states[:, 2] == 0]
north_wall_facing = north_facing

In [75]:
# Batch observation function
north_facing_idxes = np.argwhere(states[:, 2] == 0)[:, 0]
relative_n_facing_wall_idxes = np.argwhere(states[north_facing_idxes][:, 0] == 1)[:, 0]
north_facing_wall_idxes = north_facing_idxes[relative_n_facing_wall_idxes]

east_facing_idxes = np.argwhere(states[:, 2] == 1)[:, 0]
relative_e_facing_wall_idxes = np.argwhere(states[east_facing_idxes][:, 1] == size - 2)[:, 0]
east_facing_wall_idxes = east_facing_idxes[relative_e_facing_wall_idxes]

south_facing_idxes = np.argwhere(states[:, 2] == 2)[:, 0]
relative_s_facing_wall_idxes = np.argwhere(states[south_facing_idxes][:, 0] == size - 2)[:, 0]
south_facing_wall_idxes = south_facing_idxes[relative_s_facing_wall_idxes]

west_facing_idxes = np.argwhere(states[:, 2] == 3)[:, 0]
relative_w_facing_wall_idxes = np.argwhere(states[west_facing_idxes][:, 1] == 1)[:, 0]
west_facing_wall_idxes = west_facing_idxes[relative_w_facing_wall_idxes]

relative_w_facing_green_idxes = np.argwhere(states[west_facing_wall_idxes][:, 0] == 1)
relative_w_facing_blue_idxes = np.argwhere(states[west_facing_wall_idxes][:, 0] != 1)
green_facing_wall_idxes = west_facing_wall_idxes[relative_w_facing_green_idxes]
blue_facing_wall_idxes = west_facing_wall_idxes[relative_w_facing_blue_idxes]

states[blue_facing_wall_idxes]

array([[[2, 1, 3]]])

In [77]:
# Batch transition function
full_direction_mapping = np.concatenate((direction_mapping, np.zeros(direction_mapping.shape[0], dtype=np.int16)[:, np.newaxis]), axis=-1)

directions = np.arange(4)
next_states = states.copy()

forward_states = np.argwhere((actions == 0))[:, 0]

def forward(states: np.ndarray, f_states: np.ndarray, direction: int) -> np.ndarray:
    relative_d_forward_states = np.argwhere(states[f_states][:, 2] == direction)
    d_forward_states = forward_states[relative_d_forward_states][:, 0]
    return d_forward_states

for d in directions:
    d_forward_states = forward(states, forward_states, d)
    next_states[d_forward_states] += full_direction_mapping[d]

right_states = np.argwhere((actions == 1))[:, 0]
next_right_states = next_states[right_states]
next_right_states[:, 2] = (states[right_states][:, 2] + 1) % 4
next_states[right_states] = next_right_states

left_states = np.argwhere((actions == 2))[:, 0]
next_left_states = next_states[left_states]
next_left_states[:, 2] = (states[left_states][:, 2] - 1) % 4
next_states[left_states] = next_left_states



In [94]:
right_states = np.argwhere((actions == 1))[:, 0]
next_right_states = next_states[right_states]
next_right_states[:, 2] = (states[right_states][:, 2] + 1) % 4
next_states[right_states] = next_right_states

In [95]:
left_states = np.argwhere((actions == 2))[:, 0]
next_left_states = next_states[left_states]
next_left_states[:, 2] = (states[left_states][:, 2] - 1) % 4
next_states[left_states] = next_left_states

(array([[4, 7, 2],
        [3, 3, 3],
        [1, 1, 0]]),
 array([[4, 7, 1],
        [3, 3, 2],
        [1, 1, 3]]))

In [99]:
np.maximum(np.minimum(next_states, state_max), state_min)

array([[1, 3, 0],
       [4, 7, 2],
       [7, 3, 2],
       [2, 1, 3],
       [3, 3, 3],
       [4, 4, 0],
       [1, 1, 0],
       [1, 4, 0]])

In [102]:
a = np.array([[1, 2], [3, 2]])
b = np.array([3, 2])

In [104]:
(a == b).min(axis=-1)

array([False,  True])

In [8]:
np.concatenate((np.eye(5), np.zeros((1, 5))), axis=0)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])